In [ ]:
import numpy as np 
import pandas as pd 
import xarray as xr 
import cartopy 
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import scipy 
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, make_union
import warnings
import seaborn as sns
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.offsetbox as offsetbox
import seaborn.objects as so
from seaborn import axes_style
import matplotlib as mpl

import pickle

from global_var_and_func import *


warnings.filterwarnings("ignore")
#from sklearn_xarray import Stacker, Select

In [ ]:
root_path='/data/keeling/a/rytam2/ccf_model_spread/data/preprocessed/'
kernel_path='/data/keeling/a/rytam2/a/met_kernels/gcms/'
obs_kernel_path='/data/keeling/a/rytam2/a/met_kernels/obs/'

In [ ]:
# Load Kernels 
kernels_list = glob.glob(kernel_path+'*.nc');

dict_sst_kernels = {}
dict_eis_kernels = {}
dict_tadv_kernels = {}
dict_rh_kernels = {}
dict_omega_kernels = {}
dict_ws_kernels = {}

# Get dRdxi from files
for filename in kernels_list:
    kernel=xr.open_mfdataset(filename)
    j = kernels_list.index(filename)
    modelname=model_list[j]
    dict_sst_kernels["%s"%modelname] = kernel.isel(i=0).dRdxi
    dict_eis_kernels["%s"%modelname] = kernel.isel(i=1).dRdxi
    dict_tadv_kernels["%s"%modelname] = kernel.isel(i=2).dRdxi
    dict_rh_kernels["%s"%modelname] = kernel.isel(i=3).dRdxi
    dict_omega_kernels["%s"%modelname] = kernel.isel(i=4).dRdxi
    dict_ws_kernels["%s"%modelname] = kernel.isel(i=5).dRdxi
    
# Define new coords 
coords = {'lat':(['latitude'],kernel.lat.values),
         'lon':(['longitude'],kernel.lon.values)}

# Create kernel xarray variables 
sst_kernels = xr.Dataset(data_vars=dict_sst_kernels,coords=coords);
eis_kernels = xr.Dataset(data_vars=dict_eis_kernels,coords=coords);
tadv_kernels = xr.Dataset(data_vars=dict_tadv_kernels,coords=coords);
rh_kernels = xr.Dataset(data_vars=dict_rh_kernels,coords=coords);
omega_kernels = xr.Dataset(data_vars=dict_omega_kernels,coords=coords);
ws_kernels = xr.Dataset(data_vars=dict_ws_kernels,coords=coords);

# Rename to avoid duplicate dimensions
sst_kernels = sst_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
eis_kernels = eis_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
tadv_kernels = tadv_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
rh_kernels = rh_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
omega_kernels = omega_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
ws_kernels = ws_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});

In [ ]:
obskernels_list = glob.glob(obs_kernel_path+'*.nc');

for filename in obskernels_list:
    kernel=xr.open_mfdataset(filename)

dict_sst_kernels = {}
dict_eis_kernels = {}
dict_tadv_kernels = {}
dict_rh_kernels = {}
dict_omega_kernels = {}
dict_ws_kernels = {}

obskernel_namelist = ['ceres','isccp','modis','patmosx']

# Get dRdxi from files
for filename in obskernels_list:
    kernel=xr.open_mfdataset(filename)
    j = obskernels_list.index(filename)
    obsname=obskernel_namelist[j]
    dict_sst_kernels["%s"%obsname] = kernel.isel(i=0).dRdxi
    dict_eis_kernels["%s"%obsname] = kernel.isel(i=1).dRdxi
    dict_tadv_kernels["%s"%obsname] = kernel.isel(i=2).dRdxi
    dict_rh_kernels["%s"%obsname] = kernel.isel(i=3).dRdxi
    dict_omega_kernels["%s"%obsname] = kernel.isel(i=4).dRdxi
    dict_ws_kernels["%s"%obsname] = kernel.isel(i=5).dRdxi
    
# Define new coords 
coords = {'lat':(['latitude'],kernel.latitude.values),
         'lon':(['longitude'],kernel.longitude.values)}

# Create kernel xarray variables 
sst_obs_kernels = xr.Dataset(data_vars=dict_sst_kernels,coords=coords);
eis_obs_kernels = xr.Dataset(data_vars=dict_eis_kernels,coords=coords);
tadv_obs_kernels = xr.Dataset(data_vars=dict_tadv_kernels,coords=coords);
rh_obs_kernels = xr.Dataset(data_vars=dict_rh_kernels,coords=coords);
omega_obs_kernels = xr.Dataset(data_vars=dict_omega_kernels,coords=coords);
ws_obs_kernels = xr.Dataset(data_vars=dict_ws_kernels,coords=coords);

# Rename to avoid duplicate dimensions
sst_obs_kernels = sst_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'}).drop_vars(["longitude","latitude","i"])
eis_obs_kernels = eis_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'}).drop_vars(["longitude","latitude","i"])
tadv_obs_kernels = tadv_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'}).drop_vars(["longitude","latitude","i"])
rh_obs_kernels = rh_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'}).drop_vars(["longitude","latitude","i"])
omega_obs_kernels = omega_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'}).drop_vars(["longitude","latitude","i"])
ws_obs_kernels = ws_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'}).drop_vars(["longitude","latitude","i"])

In [ ]:
kernels = [sst_kernels,eis_kernels,tadv_kernels,rh_kernels,omega_kernels,ws_kernels,\
           sst_obs_kernels,eis_obs_kernels,tadv_obs_kernels,rh_obs_kernels,omega_obs_kernels,ws_obs_kernels]

In [ ]:
with open('kernel.pkl', 'wb') as fp:
    pickle.dump(kernels, fp)
    print('dictionary saved successfully to file')